# Read Image data into memory

In [1]:
import time
import getpass
import warnings
warnings.filterwarnings('ignore')

username = getpass.getuser()
start_time = time.time()
#Where is the data?
if username == "bdutt":
    baseDir="/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw"
elif username =="cgarg":
    baseDir="/Users/cgarg/Documents/charu/intuit/code/face/data/lfw"
flow = "main"
debug = False

%run "Load Data.ipynb"

filenames = []
labels = []

def collectFileNamesAndLabels(filename):
    filenames.append(filename)
    result = filename.split("/") 
    labels.append(result[len(result)-2]) #Get the label name from filename
    
doForEachFile(collectFileNamesAndLabels,baseDir, 10)
print("Total File name collection time = %s seconds" % (time.time() - start_time))
print ("Files collected : ",len(filenames))

Total File name collection time = 1.7744061946868896 seconds
Files collected :  6


In [2]:
start_time = time.time()
imagesList = []
doForEachFileNames(readImage, filenames, imagesList)
print ("Images data collected : ",len(imagesList))
print("Total data collection time = %s seconds" % (time.time() - start_time))

Images data collected :  6
Total data collection time = 0.033499956130981445 seconds


In [3]:
#%run "TransferLearningWithCNN.ipynb"

Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw//Doris_Roberts/Doris_Roberts_0001.jpg'

# Generic Multi threading code

Used later

In [4]:
import threading
import sys
import traceback
import time

class myThread (threading.Thread):
    def __init__(self, threadName, index, worker):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.index = index
        self.worker = worker
    def run(self):
        print('^',self.threadName, ', ', end='')
        try:
            self.worker(self.threadName, self.index)
        except:
            e = sys.exc_info()[0]
            print("Exception in thread: ",self.threadName,", ", e)
            traceback.print_exc()
        print('V',self.threadName, end='')
        

def runThreads(nThreads, worker):
    threads = []
    for i in range(nThreads):
        try:
            t = myThread("Thread-"+str(i), i , worker)
            threads.append(t)
            t.start()
        except:
            e = sys.exc_info()[0]
            print("Error: unable to start thread: ", e)
            traceback.print_exc()

    time.sleep(1)

    for t in threads:
        t.join()

In [5]:
start_time = time.time()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


Y = pd.DataFrame(labels, columns = ['name'])
sizeOfYLabel = len(Y)
oneDImageList = []
showOneImage = False

imageListLock = threading.Lock()

def flattenImage(image, imageList):
    # Convert RGB to single number
    global showOneImage
    if showOneImage:
        plt.imshow(image, cmap="Greys")
        plt.show()
    gray = rgb2gray(image)
    if showOneImage:
        plt.imshow(gray, cmap="Greys")
        plt.show()
        showOneImage = False
    imageList.append(gray.flatten())

nThreads = 10
splitImagesList = np.array_split(imagesList, nThreads)
splitLabelsList = np.array_split(labels, nThreads)
labels = []

def imageFlatteningWorker(threadName, index):
    global splitImagesList
    global imageListLock
    imageList = []
    for image in splitImagesList[index]:
        flattenImage(image, imageList)
    imageListLock.acquire()
    global oneDImageList
    oneDImageList.extend(imageList)
    global labels
    labels.extend(splitLabelsList[index])
    imageListLock.release()

runThreads(nThreads, imageFlatteningWorker)
    
del imagesList

X = pd.DataFrame(oneDImageList)
del oneDImageList
print("X shape", X.shape)
print("label count", len(labels))
print("Total dataframe creation time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2 , ^ Thread-3 , ^^ Thread-4 ,  ^Thread-5 ,  ^V Thread-0 Thread-7 , V Thread-7Thread-6V^ Thread-8 , V Thread-3 Thread-1 , VVV Thread-2 Thread-4 Thread-6V Thread-8^V Thread-9 , V Thread-9 Thread-5X shape (6, 62500)
label count 6
Total dataframe creation time = 4.43583083152771 seconds


In [6]:
#Y['name'].value_counts()

In [7]:
start_time = time.time()
maxFrequency = 21

# We upsample/downsample to have equal quantity of all classes

#X_sample = pd.DataFrame(columns = X.columns)
#Y_sample = pd.DataFrame(columns = Y.columns)
X_sampleList = []
Y_sampleList = []

XY_sampleLock = threading.Lock()

def equalSampleLabel(label):
    global X_sampleList
    global Y_sampleList
    global XY_sampleLock
    dfa = X[Y['name']== label]
    dfasample = dfa.sample(n=maxFrequency,replace=True)
    dfb = Y[Y['name']== label]
    dfbsample = dfb.sample(n=maxFrequency,replace=True)
    print('.', end='')
    XY_sampleLock.acquire()
    print('-', end='') 
    X_sampleList.append(dfasample)
    Y_sampleList.append(dfbsample)
    XY_sampleLock.release()
    print('+', end='')



labels = Y['name'].unique()

nThreads = 10
labelsArray = np.array_split(labels, nThreads)

def worker(threadName, index):
    for label in labelsArray[index]:
        #print(threadName, label)
        equalSampleLabel(label)

runThreads(nThreads, worker)

X_sample = pd.concat(X_sampleList)
Y_sample = pd.concat(Y_sampleList)
print(X_sample.shape)
print(Y_sample.shape)
X = X_sample
Y = Y_sample
# print(X_sample)
print("Total equal sampling time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2 , ^ Thread-3 , ^ Thread-4 , ^ Thread-5 , ^ Thread-6 , V Thread-6^. ^Thread-7. -, ^ V Thread-7Thread-8 , V Thread-8.+V Thread-1 -+V Thread-0Thread-9 , V Thread-9-+V Thread-2.-+V Thread-4.-+V Thread-3.-+V Thread-5(126, 62500)
(126, 1)
Total equal sampling time = 1.1357238292694092 seconds


In [8]:
start_time = time.time()
sizeOfX = X_sample.shape[1]
#%run "DNN.ipynb"

print("Total DNN time = %s seconds" % (time.time() - start_time))

Shape of X to DNN (126, 62500)
Shape of Y to DNN (126, 1)
Shape of Y post OneHot encoding (126, 6)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                3125050   
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 156       
Total params: 3,126,481
Trainable params: 3,126,481
Non-trainable params: 0
__________________________________

126/126 [==============================] - 0s 1ms/step - loss: 0.2628 - categorical_accuracy: 0.7778
Epoch 57/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2684 - categorical_accuracy: 0.7381
Epoch 58/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2641 - categorical_accuracy: 0.7698
Epoch 59/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2681 - categorical_accuracy: 0.7698
Epoch 60/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2603 - categorical_accuracy: 0.7857
Epoch 61/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2374 - categorical_accuracy: 0.8016
Epoch 62/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2320 - categorical_accuracy: 0.8651
Epoch 63/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2456 - categorical_accuracy: 0.8175
Epoch 64/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2306 - categorical_accu

126/126 [==============================] - 0s 1ms/step - loss: 0.1930 - categorical_accuracy: 0.8571
Epoch 128/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1981 - categorical_accuracy: 0.8651
Epoch 129/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1812 - categorical_accuracy: 0.8492
Epoch 130/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1785 - categorical_accuracy: 0.8968
Epoch 131/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2042 - categorical_accuracy: 0.8571
Epoch 132/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2259 - categorical_accuracy: 0.7698
Epoch 133/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1871 - categorical_accuracy: 0.8651
Epoch 134/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1900 - categorical_accuracy: 0.8651
Epoch 135/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1947 - categori

126/126 [==============================] - 0s 1ms/step - loss: 0.1817 - categorical_accuracy: 0.8730
Epoch 199/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1639 - categorical_accuracy: 0.8810
Epoch 200/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1489 - categorical_accuracy: 0.9048
Epoch 201/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1519 - categorical_accuracy: 0.8968
Epoch 202/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1699 - categorical_accuracy: 0.8651
Epoch 203/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1548 - categorical_accuracy: 0.8889
Epoch 204/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1672 - categorical_accuracy: 0.9048
Epoch 205/500
126/126 [==============================] - 0s 1ms/step - loss: 0.2210 - categorical_accuracy: 0.8492
Epoch 206/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1464 - categori

126/126 [==============================] - 0s 1ms/step - loss: 0.1698 - categorical_accuracy: 0.8571
Epoch 270/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1373 - categorical_accuracy: 0.9206
Epoch 271/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1518 - categorical_accuracy: 0.9286
Epoch 272/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1695 - categorical_accuracy: 0.8651
Epoch 273/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1410 - categorical_accuracy: 0.9048
Epoch 274/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1583 - categorical_accuracy: 0.8571
Epoch 275/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1600 - categorical_accuracy: 0.8968
Epoch 276/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1223 - categorical_accuracy: 0.9524
Epoch 277/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1315 - categori

126/126 [==============================] - 0s 1ms/step - loss: 0.1328 - categorical_accuracy: 0.9444
Epoch 341/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1293 - categorical_accuracy: 0.9127
Epoch 342/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1386 - categorical_accuracy: 0.9048
Epoch 343/500
126/126 [==============================] - 0s 2ms/step - loss: 0.1781 - categorical_accuracy: 0.8571
Epoch 344/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1385 - categorical_accuracy: 0.8492
Epoch 345/500
126/126 [==============================] - 0s 1ms/step - loss: 0.0974 - categorical_accuracy: 0.9603
Epoch 346/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1512 - categorical_accuracy: 0.8730
Epoch 347/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1409 - categorical_accuracy: 0.8968
Epoch 348/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1339 - categori

126/126 [==============================] - 0s 1ms/step - loss: 0.1139 - categorical_accuracy: 0.9286
Epoch 412/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1381 - categorical_accuracy: 0.8730
Epoch 413/500
126/126 [==============================] - 0s 962us/step - loss: 0.1310 - categorical_accuracy: 0.9048
Epoch 414/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1186 - categorical_accuracy: 0.8968
Epoch 415/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1201 - categorical_accuracy: 0.8968
Epoch 416/500
126/126 [==============================] - 0s 976us/step - loss: 0.1403 - categorical_accuracy: 0.9127
Epoch 417/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1189 - categorical_accuracy: 0.9206
Epoch 418/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1210 - categorical_accuracy: 0.9286
Epoch 419/500
126/126 [==============================] - 0s 1ms/step - loss: 0.0810 - cate

126/126 [==============================] - 0s 1ms/step - loss: 0.1284 - categorical_accuracy: 0.8810
Epoch 483/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1094 - categorical_accuracy: 0.9127
Epoch 484/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1081 - categorical_accuracy: 0.9048
Epoch 485/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1043 - categorical_accuracy: 0.9286
Epoch 486/500
126/126 [==============================] - 0s 2ms/step - loss: 0.0966 - categorical_accuracy: 0.9524
Epoch 487/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1204 - categorical_accuracy: 0.9286
Epoch 488/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1093 - categorical_accuracy: 0.8810
Epoch 489/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1274 - categorical_accuracy: 0.8968
Epoch 490/500
126/126 [==============================] - 0s 1ms/step - loss: 0.1379 - categori

In [9]:
print(X_sample.shape)


(126, 62500)


In [12]:
# print(Y)
enc.get_feature_names(['name'])
print(enc.get_feature_names(['name']).shape)

(6,)


In [11]:
import numpy as np
import tensorflow as tf

Ypred = pd.DataFrame( columns = ['sampleId', "classAsIndex", "classAsString", "actualClass"])
for i in range(300):
    prediction = model.predict(X_sample.iloc[i:i+1,:],verbose=0)
    class_labels = np.argmax(prediction[0], axis=0)
    Ypred.at[i, 'sampleId']  = i
    Ypred.at[i, 'classAsIndex']  = class_labels
    Ypred.at[i, 'classAsString']  = enc.get_feature_names(['name'])[class_labels]
    Ypred.at[i, 'actualClass']  = Y_sample.iloc[i].values
    #print(class_labels, enc.get_feature_names(['name'])[class_labels]," Names of class :: ", Y_sample.iloc[i].values)

from IPython.display import display, HTML
display(HTML(Ypred.to_html()))

IndexError: list index out of range

In [ ]:
class_mapping = { i : labels[i] for i in range(0, len(labels) ) }
print(class_mapping)

In [ ]:
# start_time = time.time()
sizeOfX = X_sample.shape[1]
print(X_sample.shape)
%run "TrasnferLearning-New.ipynb"

print("Total transfer learning with CNN time = %s seconds" % (time.time() - start_time))